In [1]:
# Load a spacy model and chekc if it has ner
import spacy
nlp=spacy.load('en_core_web_sm')



/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
ner=nlp.get_pipe("ner")

In [3]:
# training data
TRAIN_DATA = [
              ("Walmart is a leading e-commerce company", {"entities": [(0, 7, "ORG")]}),
              ("I reached Chennai yesterday.", {"entities": [(19, 28, "GPE")]}),
              ("I recently ordered a book from Amazon", {"entities": [(24,32, "ORG")]}),
              ("I was driving a BMW", {"entities": [(16,19, "PRODUCT")]}),
              ("I ordered this from ShopClues", {"entities": [(20,29, "ORG")]}),
              ("Fridge can be ordered in Amazon ", {"entities": [(0,6, "PRODUCT")]}),
              ("I bought a new Washer", {"entities": [(16,22, "PRODUCT")]}),
              ("I bought a old table", {"entities": [(16,21, "PRODUCT")]}),
              ("I bought a fancy dress", {"entities": [(18,23, "PRODUCT")]}),
              ("I rented a camera", {"entities": [(12,18, "PRODUCT")]}),
              ("I rented a tent for our trip", {"entities": [(12,16, "PRODUCT")]}),
              ("I rented a screwdriver from our neighbour", {"entities": [(12,22, "PRODUCT")]}),
              ("I repaired my computer", {"entities": [(15,23, "PRODUCT")]}),
              ("I got my clock fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("I got my truck fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("Flipkart started it's journey from zero", {"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Max", {"entities": [(24,27, "ORG")]}),
              ("Flipkart is recognized as leader in market",{"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Swiggy", {"entities": [(24,29, "ORG")]})
              ]

In [4]:
# Adding labels to the `ner`

for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [5]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [6]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        #Updating model with each iterating text
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)
        print("Losses", losses)
    
    
          
    
    

/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I rented a screwdriver from our neighbour" with entities "[(12, 22, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I recently ordered a book from Amazon" with entities "[(24, 32, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I got my truck fixed" with entities "[(16, 21, 'PRODUCT')]". Use `spacy.traini

Losses {'ner': 0.28672915060582227}
Losses {'ner': 0.8060134709813269}
Losses {'ner': 8.039514235067657}
Losses {'ner': 9.78121932773439}
Losses {'ner': 11.864510696596978}
Losses {'ner': 5.84704533109277}
Losses {'ner': 5.855769439655972}
Losses {'ner': 7.528628890321555}
Losses {'ner': 7.532154572682257}
Losses {'ner': 11.587715741026061}
Losses {'ner': 0.40468386744049667}
Losses {'ner': 0.4156463386986502}
Losses {'ner': 1.3163882569069472}
Losses {'ner': 6.99105037425443}
Losses {'ner': 7.067504473637176}
Losses {'ner': 1.2305979743166482}
Losses {'ner': 2.853126776095351}
Losses {'ner': 5.1202419944920035}
Losses {'ner': 6.240172698635677}
Losses {'ner': 9.239245030548503}
Losses {'ner': 0.2699293374085858}
Losses {'ner': 3.610151279643391}
Losses {'ner': 4.9962906933262285}
Losses {'ner': 4.996304539839534}
Losses {'ner': 5.016959845112135}
Losses {'ner': 1.3607585909383602}
Losses {'ner': 1.3611101242366987}
Losses {'ner': 2.911611742620812}
Losses {'ner': 3.0311225936245223}
L

In [7]:
# Testing the model
doc = nlp("I was driving a Alto")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Alto', 'PRODUCT')]
